In [19]:
import sys
sys.path.append('..')

In [20]:
import numpy as np
import os
import tensorflow as tf
import pandas as pd
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import initializers
from preprocessing import parse_aug_fn, parse_fn

In [23]:
train_split, valid_split = ['train[:90%]', 'train[90%:]']
train_data, info = tfds.load('cifar10', split=train_split, with_info=True)
vaild_data = tfds.load('cifar10', split=valid_split)
test_data = tfds.load('cifar10', split='test')

print(train_data)


<PrefetchDataset shapes: {id: (), image: (32, 32, 3), label: ()}, types: {id: tf.string, image: tf.uint8, label: tf.int64}>


In [24]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
batch_size = 64
train_num = int(info.splits['train'].num_examples / 10 )* 9

train_data = train_data.shuffle(train_num)
train_data = train_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
train_data = train_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

vaild_data = vaild_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
vaild_data = vaild_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

test_data = test_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
test_data = test_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)



In [25]:
def bulid_and_train_model(run_name,init):
    inputs = keras.Input(shape=(32, 32, 3))
    x = layers.Conv2D(64, (3, 3), kernel_initializer=init, activation='relu')(inputs)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(128, (3, 3), kernel_initializer=init, activation='relu')(x)
    x = layers.Conv2D(256, (3, 3), kernel_initializer=init, activation='relu')(x)
    x = layers.Conv2D(128, (3, 3), kernel_initializer=init, activation='relu')(x)
    x = layers.Conv2D(64, (3, 3), kernel_initializer=init, activation='relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, kernel_initializer=init, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(10, kernel_initializer=init, activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    logfiles = 'lab5-logs/{}-{}'.format(run_name, init.__class__.__name__)
    model_mbk = keras.callbacks.TensorBoard(logfiles, histogram_freq=1)
    model_file = logfiles + '/model'.format(run_name)
    model_mckp = keras.callbacks.ModelCheckpoint(model_file, save_best_only=True,model='max',monitor='val_categorical_accuracy')

    model.compile(keras.optimizers.Adam(), loss=keras.losses.CategoricalCrossentropy(), metrics=[keras.metrics.CategoricalAccuracy()])

    model.fit(train_data, epochs=50, validation_data=vaild_data, callbacks=[model_mbk, model_mckp])

In [26]:
session_num = 1
model_dir = 'lab5-logs/models/'
weights_initializers_list = [initializers.RandomNormal(), initializers.GlorotNormal(), initializers.HeNormal()]
for init in weights_initializers_list:
    print('---runing session %d with init %s---' % (session_num, init.__class__.__name__))
    run_name = 'run-%d' % session_num
    bulid_and_train_model(run_name, init)
    session_num += 1


---runing session 1 with init RandomNormal---
Epoch 1/50
  1/704 [..............................] - ETA: 0s - loss: 2.4027 - categorical_accuracy: 0.1094WARNING:tensorflow:From c:\Users\Admin\miniconda3\envs\tf2\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


Instructions for updating:
use `tf.profiler.experimental.stop` instead.


703/704 [============================>.] - ETA: 0s - loss: 1.9891 - categorical_accuracy: 0.2502WARNING:tensorflow:From c:\Users\Admin\miniconda3\envs\tf2\lib\site-packages\tensorflow\python\training\tracking\tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


...

704/704 [==============================] - 58s 82ms/step - loss: 0.8346 - categorical_accuracy: 0.7221 - val_loss: 0.6631 - val_categorical_accuracy: 0.7788
Epoch 27/50
704/704 [==============================] - 57s 81ms/step - loss: 0.8105 - categorical_accuracy: 0.7286 - val_loss: 0.7636 - val_categorical_accuracy: 0.7534
Epoch 28/50
704/704 [==============================] - 61s 87ms/step - loss: 0.8165 - categorical_accuracy: 0.7281 - val_loss: 0.6945 - val_categorical_accuracy: 0.7606
Epoch 29/50
704/704 [==============================] - 59s 83ms/step - loss: 0.8068 - categorical_accuracy: 0.7305 - val_loss: 0.7117 - val_categorical_accuracy: 0.7724
Epoch 30/50
704/704 [==============================] - 58s 82ms/step - loss: 0.7917 - categorical_accuracy: 0.7341 - val_loss: 0.6709 - val_categorical_accuracy: 0.7730
Epoch 31/50
704/704 [==============================] - 58s 82ms/step - loss: 0.7885 - categorical_accuracy: 0.7366 - val_loss: 0.6787 - val_categorical_accuracy: 0.775

INFO:tensorflow:Assets written to: lab5-logs/run-1-RandomNormal\model\assets


704/704 [==============================] - 59s 84ms/step - loss: 0.7594 - categorical_accuracy: 0.7465 - val_loss: 0.6605 - val_categorical_accuracy: 0.7820
Epoch 34/50
704/704 [==============================] - 59s 83ms/step - loss: 0.7636 - categorical_accuracy: 0.7464 - val_loss: 0.6498 - val_categorical_accuracy: 0.7818
Epoch 35/50
704/704 [==============================] - 59s 83ms/step - loss: 0.7476 - categorical_accuracy: 0.7496 - val_loss: 0.6625 - val_categorical_accuracy: 0.7780
Epoch 36/50
703/704 [============================>.] - ETA: 0s - loss: 0.7540 - categorical_accuracy: 0.7498INFO:tensorflow:Assets written to: lab5-logs/run-1-RandomNormal\model\assets


INFO:tensorflow:Assets written to: lab5-logs/run-1-RandomNormal\model\assets


704/704 [==============================] - 59s 84ms/step - loss: 0.7540 - categorical_accuracy: 0.7499 - val_loss: 0.6568 - val_categorical_accuracy: 0.7900
Epoch 37/50
704/704 [==============================] - 59s 83ms/step - loss: 0.7403 - categorical_accuracy: 0.7535 - val_loss: 0.6743 - val_categorical_accuracy: 0.7790
Epoch 38/50
704/704 [==============================] - 59s 83ms/step - loss: 0.7336 - categorical_accuracy: 0.7578 - val_loss: 0.6539 - val_categorical_accuracy: 0.7812
Epoch 39/50
704/704 [==============================] - 59s 84ms/step - loss: 0.7273 - categorical_accuracy: 0.7600 - val_loss: 0.6661 - val_categorical_accuracy: 0.7764
Epoch 40/50
704/704 [==============================] - 61s 87ms/step - loss: 0.7226 - categorical_accuracy: 0.7623 - val_loss: 0.6655 - val_categorical_accuracy: 0.7854
Epoch 41/50
704/704 [==============================] - 61s 87ms/step - loss: 0.7169 - categorical_accuracy: 0.7627 - val_loss: 0.6495 - val_categorical_accuracy: 0.786